In [1]:
#%load_ext lab_black

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import pandas as pd
import numpy as np
import os
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

credentials = service_account.Credentials.from_service_account_file(
    "/Users/miguelcouto/Downloads/zattoo-dataeng-e5f45785174f.json"
)

project_id = "zattoo-dataeng"
client = bigquery.Client(credentials=credentials, project=project_id)

In [3]:
sql_calcs = """
select mt.transaction_id,
       mt.zuid,
       mt.payment_method,
       mt.sku,
       mt.type_of_transaction,
       case
           when mt.country_name = 'Germany' then 'Germany'
           when mt.country_name = 'Austria' then 'Austria'
           else 'Switzerland' end as country_name,
       mt.new_booking_net_CHF,
       mt.renewal_booking_net_CHF,
       mt.new_booking_net_EUR,
       mt.renewal_booking_net_EUR,
       mt.transaction_date,
       mt.term_start,
       mt.term_end,
       product_service_view.zuya_account_permission as product_group_finance,
       mt.vat_eur,
       mt.vat_chf
from b2c_middleware.middlelayer_transactions mt
         LEFT JOIN b2c_middleware_import.product_product_view AS product_product_view
                   ON mt.sku = product_product_view.sku
                       AND DATE(mt.transaction_date) = product_product_view.inserted_at_date
         LEFT JOIN b2c_middleware_import.product_subscriptionproduct_view AS product_subscriptionproduct_view
                   ON product_product_view.id = product_subscriptionproduct_view.id
                       AND
                      product_product_view.inserted_at_date = product_subscriptionproduct_view.inserted_at_date
         LEFT JOIN b2c_middleware_import.product_offer_view AS product_offer_view
                   ON product_subscriptionproduct_view.offer_id = product_offer_view.id
                       AND product_subscriptionproduct_view.inserted_at_date = product_offer_view.inserted_at_date
         LEFT JOIN b2c_middleware_import.product_service_view AS product_service_view
                   ON product_offer_view.service_id = product_service_view.id
                       AND product_offer_view.inserted_at_date = product_service_view.inserted_at_date
where true
  and mt.app_shop_id = 'amazon'
  and mt.transaction_date >= '2022-02-01 00:00:00'
  and mt.transaction_date <= '2022-02-28 23:59:59'
-- NEW LINE! added on 2022-04-27
 -- and mt.type_of_transaction != 'refund'
           """

In [4]:
## prepare dataframe
df = pandas_gbq.read_gbq(sql_calcs, project_id=project_id, progress_bar_type=None)
type_trans_lst = ["new_sale", "renewal", "refund"]

In [5]:
parse_dates = ["term_start", "term_end"]
mt_df = df[df["type_of_transaction"].isin(type_trans_lst)].copy()

for date in parse_dates:
    mt_df[date] = pd.to_datetime(mt_df[date]).dt.tz_convert(None)

In [6]:
## calculate total_booking_net columns
mt_df["total_booking_net_CHF"] = (
        mt_df["new_booking_net_CHF"] + mt_df["renewal_booking_net_CHF"]
)
mt_df["total_booking_net_EUR"] = (
        mt_df["new_booking_net_EUR"] + mt_df["renewal_booking_net_EUR"]
)

In [7]:
## drop unnecessary columns
mt_df.drop(
    [
        "new_booking_net_EUR",
        "new_booking_net_CHF",
        "renewal_booking_net_CHF",
        "renewal_booking_net_EUR",
    ],
    axis=1,
    inplace=True,
)

In [8]:
## calculate product_term_length_months
mt_df["product_term_length_months"] = (
        (mt_df["term_end"].dt.year - mt_df["term_start"].dt.year) * 12
        + (mt_df["term_end"].dt.month - mt_df["term_start"].dt.month)
        + 1
)

In [9]:
mt_df["product_term_length_months"][mt_df["product_term_length_months"] < 0] = 0

In [10]:
## replacing product_term_length_months for exceptions where value is 2 instead of 1
shorter_subs = (mt_df['term_end'] - mt_df['term_start']).dt.days <= 30
shorter_subs_replacer = mt_df[shorter_subs][
    (mt_df["product_term_length_months"] == 2) & ((mt_df['term_end']).dt.day == 1)].index.to_list()
mt_df["product_term_length_months"].loc[shorter_subs_replacer] = 1

In [11]:
## reseting index
mt_df = mt_df.reindex(mt_df.index.repeat(mt_df["product_term_length_months"]))

In [12]:
## add revenue_month_number
mt_df["revenue_month_number"] = mt_df.groupby(["transaction_id"]).cumcount() + 1

In [13]:
## add revenue_month_date
mt_df["revenue_month_date"] = mt_df["term_start"].to_numpy().astype("datetime64[M]")
mt_df["revenue_month_date"][mt_df["type_of_transaction"] == "refund"] = (
    mt_df["transaction_date"].to_numpy().astype("datetime64[M]")
)

mt_df["revenue_month_date"] = mt_df.apply(
    lambda x: x["revenue_month_date"]
              + pd.offsets.MonthEnd(x["revenue_month_number"])
              + pd.offsets.MonthBegin(-1),
    axis=1,
)

## NEW LINE! commenting out as we now have product_service_view.zuya_account_permission available
# mt_df["product_group_finance"] = np.nan

mt_df["product_term_length"] = mt_df["revenue_month_date"].apply(
    lambda t: pd.Period(t, freq="S").days_in_month
)

mt_df["product_term_length"][mt_df["term_start"] > mt_df["revenue_month_date"]] = (
                                                                                          mt_df[
                                                                                              "term_start"].dt.daysinmonth -
                                                                                          mt_df["term_start"].dt.day
                                                                                  ) + 1

In [14]:
## reseting index
mt_df.reset_index(drop=True, inplace=True)

In [15]:
## set active_sub_month_end = 1 by default
mt_df["active_sub_month_end"] = 1

## get last indices of each transaction_id group
last_idxs = (
        len(mt_df)
        - np.unique(
    mt_df.transaction_id.values[::-1],
    return_index=1,
)[1]
        - 1
)

mt_df["active_sub_month_end"].iloc[last_idxs] = 0

## mark all one transaction subscriptions as 1
mt_df["active_sub_month_end"][
    (mt_df["revenue_month_number"] <= 1) & (mt_df["product_term_length_months"] <= 1)
    ] = 1

## mark all refund transactions as -1
mt_df["active_sub_month_end"][mt_df["type_of_transaction"] == "refund"] = -1

## mark all subscriptions with only 1 transaction as i
mt_df["active_sub_month_end"][mt_df["type_of_transaction"] == "refund"] = -1

In [16]:
## active_sub_content follows the same logic as active_sub_month_end except it doesn't count the last month
mt_df["active_sub_content"] = mt_df.active_sub_month_end

mt_df.loc[mt_df.groupby('transaction_id')['active_sub_content'].tail(2).index, 'active_sub_content'] = 0

In [17]:
## fix last position of product_term_length per transaction_id
mt_df["product_term_length"].iloc[last_idxs] = (
        mt_df["term_end"].iloc[last_idxs] -
        mt_df["revenue_month_date"].iloc[last_idxs]
).dt.days

In [18]:
## TEMP FIX to tackle dynamic term_end for refunds
mt_df["product_term_length"][mt_df["product_term_length"] < 0] = 0

In [19]:
## total_days of product_term_length per transaction_id
mt_df["total_days"] = mt_df.groupby("transaction_id")["product_term_length"].transform(
    "sum"
)

In [20]:
## get 12mo subs that didn't stay for 12 months for posterior treatment of exception
mt_df['max_revenue_month_number'] = mt_df.groupby(['transaction_id'])['revenue_month_number'].transform(max)

# mt_df[(mt_df.sku.str.contains('zattoo_web_hiq_swiss_12mo')) & (mt_df.max_revenue_month_number < 13)]

In [21]:
## NEW LINE! added on 2022-04-27
## product_term_length fix for subscriptions < 31 days (1, 3, 7 days)
mt_df['product_term_length'][mt_df.sku.str.contains('day')] = (mt_df.term_end - mt_df.term_start).dt.days

In [22]:
trx_lst = mt_df['transaction_id'][(mt_df.sku.str.contains('day')) & (mt_df['revenue_month_number'] > 1)].unique()

In [23]:
## total_days fix for subscriptions < 31 days (1, 3, 7 days)
mt_df["total_days"][(mt_df.product_term_length != mt_df.total_days) & (mt_df['product_term_length_months'] <= 1)] = mt_df["product_term_length"][
    (mt_df.product_term_length != mt_df.total_days) & (mt_df['product_term_length_months'] <= 1)]

In [24]:
mt_df[mt_df['transaction_id'] == '36062887']

,transaction_id,zuid,payment_method,sku,type_of_transaction,country_name,transaction_date,term_start,term_end,product_group_finance,vat_eur,vat_chf,total_booking_net_CHF,total_booking_net_EUR,product_term_length_months,revenue_month_number,revenue_month_date,product_term_length,active_sub_month_end,active_sub_content,total_days,max_revenue_month_number


In [25]:
## calculate total_revenue_net fields
mt_df["total_revenue_net_EUR"] = (
        mt_df["total_booking_net_EUR"] / mt_df["total_days"] * mt_df["product_term_length"]
)

mt_df["total_revenue_net_CHF"] = (
        mt_df["total_booking_net_CHF"] / mt_df["total_days"] * mt_df["product_term_length"]
)

## remove total_booking values from all lines of group except first
mt_df.loc[
    mt_df["revenue_month_number"] > 1,
    ["total_booking_net_CHF", "total_booking_net_EUR"],
] = 0.0

In [26]:
## remove VAT values from all lines of group except first
vat_cols = ["vat_chf", "vat_eur"]

for col in vat_cols:
    mt_df.loc[
        mt_df["revenue_month_number"] > 1,
        [col, "total_booking_net_EUR"],
    ] = 0.0

In [27]:
## prepare df_nocalcs for free trials and full discounts
mt_df_nocalcs = df[~df["type_of_transaction"].isin(type_trans_lst)].copy()

for date in parse_dates:
    mt_df_nocalcs[date] = pd.to_datetime(mt_df_nocalcs[date]).dt.tz_convert(None)

In [28]:
## calculate total_booking_net columns
mt_df_nocalcs["total_booking_net_CHF"] = (
        mt_df_nocalcs["new_booking_net_CHF"] + mt_df_nocalcs["renewal_booking_net_CHF"]
)
mt_df_nocalcs["total_booking_net_EUR"] = (
        mt_df_nocalcs["new_booking_net_EUR"] + mt_df_nocalcs["renewal_booking_net_EUR"]
)

In [29]:
## adding vat_percentage to calculate sales_price
mt_df['vat_eur_percentage'] = (mt_df['vat_eur'] / mt_df['total_booking_net_EUR']).astype(float, errors='ignore')
mt_df['vat_chf_percentage'] = (mt_df['vat_chf'] / mt_df['total_booking_net_CHF']).astype(float, errors='ignore')

mt_df['sales_price_eur'] = mt_df['total_booking_net_EUR'] * (
        1 + mt_df['vat_eur_percentage'])
mt_df['sales_price_chf'] = mt_df['total_booking_net_CHF'] * (
        1 + mt_df['vat_chf_percentage'])

In [30]:
## append dataframes
mt_df_final = mt_df.append(mt_df_nocalcs).reset_index(drop=True)

## convert revenue_month_date to date
mt_df_final["revenue_month_date"] = pd.to_datetime(
    mt_df_final["revenue_month_date"]
).dt.date

In [31]:
## extract list of transaction ids whose subscription float between one month and the other, and are 1, 3 or 7 days (== product_length < 31)
trx_lst_more_1month_subs = mt_df['transaction_id'][
    (mt_df["revenue_month_number"] > 1) & (mt_df.sku.str.contains('day'))]

## this logic makes sure the days are allocated correctly for these very specific subscriptions
mt_df['product_term_length'][
    (mt_df.transaction_id.isin(trx_lst_more_1month_subs)) & (mt_df.revenue_month_number == 1)] = - (
        mt_df.term_start - mt_df.revenue_month_date.shift(1)).dt.days

mt_df['product_term_length'][
    (mt_df.transaction_id.isin(trx_lst_more_1month_subs)) & (mt_df.revenue_month_number > 1)] = (
        mt_df.term_end - mt_df.revenue_month_date).dt.days

In [32]:
## fix active_sub_month_end for subscriptions with only 1 revenue month and length < 30 days
mt_df['last_day_term_end'] = (pd.to_datetime(mt_df['term_end'], format="%d", unit='d')).dt.day
mt_df['last_day_month'] = (pd.to_datetime(mt_df['term_end'], format="%d", unit='d')+ pd.offsets.MonthEnd(n=1)).dt.day

mt_df["active_sub_month_end"][mt_df.last_day_term_end < mt_df.last_day_month] = 0

In [33]:
## fix those one line transactions that still have wrong product_term_length
trx_id_counts = mt_df['transaction_id'].value_counts(sort=False)
check_length_mask = mt_df[mt_df['transaction_id'].isin(trx_id_counts.index[trx_id_counts == 1])].index.to_list()

mt_df["product_term_length"].loc[check_length_mask] = mt_df['term_end'].dt.day - mt_df.term_start.dt.day

In [34]:
## reorder dataframe
mt_df = mt_df[
    [
        "transaction_id",
        "zuid",
        "payment_method",
        "sku",
        "type_of_transaction",
        "country_name",
        "transaction_date",
        "total_booking_net_CHF",
        "total_booking_net_EUR",
        "vat_chf",
        "vat_eur",
        "term_start",
        "term_end",
        "product_term_length",
        "product_term_length_months",
        "product_group_finance",
        "revenue_month_number",
        "revenue_month_date",
        "total_revenue_net_EUR",
        "total_revenue_net_CHF",
        "sales_price_eur",
        "sales_price_chf",
        "active_sub_month_end",
    ]
]

## define BQ table schema
bq_schema = [
    {"name": "transaction_id", "type": "STRING"},
    {"name": "zuid", "type": "INTEGER"},
    {"name": "payment_method", "type": "STRING"},
    {"name": "sku", "type": "STRING"},
    {"name": "type_of_transaction", "type": "STRING"},
    {"name": "transaction_date", "type": "TIMESTAMP"},
    {"name": "country_name", "type": "STRING"},
    {"name": "total_booking_net_CHF", "type": "FLOAT"},
    {"name": "total_booking_net_EUR", "type": "FLOAT"},
    {"name": "vat_chf", "type": "FLOAT"},
    {"name": "vat_eur", "type": "FLOAT"},
    {"name": "term_start", "type": "TIMESTAMP"},
    {"name": "term_end", "type": "TIMESTAMP"},
    {"name": "product_term_length", "type": "INTEGER"},
    {"name": "product_term_length_months", "type": "INTEGER"},
    {"name": "product_group_finance", "type": "STRING"},
    {"name": "revenue_month_number", "type": "INTEGER"},
    {"name": "revenue_month_date", "type": "DATE"},
    {"name": "total_revenue_net_EUR", "type": "FLOAT"},
    {"name": "total_revenue_net_CHF", "type": "FLOAT"},
    {"name": "sales_price_eur", "type": "FLOAT"},
    {"name": "sales_price_chf", "type": "FLOAT"},
    {"name": "active_sub_month_end", "type": "INTEGER"},
]

In [182]:
## export to BQ table
pandas_gbq.to_gbq(
    dataframe=mt_df_final,
    destination_table="temp.pypayment_v2_amazon_022022",
    project_id="zattoo-dataeng",
    if_exists="replace",
    progress_bar=None,
    table_schema=bq_schema,
)

In [35]:
mt_df_final.head()

,transaction_id,zuid,payment_method,sku,type_of_transaction,country_name,transaction_date,term_start,term_end,product_group_finance,vat_eur,vat_chf,total_booking_net_CHF,total_booking_net_EUR,product_term_length_months,revenue_month_number,revenue_month_date,product_term_length,active_sub_month_end,active_sub_content,total_days,max_revenue_month_number,total_revenue_net_EUR,total_revenue_net_CHF,vat_eur_percentage,vat_chf_percentage,sales_price_eur,sales_price_chf,new_booking_net_CHF,renewal_booking_net_CHF,new_booking_net_EUR,renewal_booking_net_EUR
0,34833988_4,29807511,amazon,zattoo_amazon_firetv_ultimate_austrian,renewal,Austria,2022-02-20 09:16:19,2022-02-20 09:16:19,2022-03-23 09:16:18,base_ultimate,2.998333,3.131579,11.089777,12.491667,2.0,1.0,2022-02-01,9.0,1.0,0.0,31.0,2.0,3.626613,3.219613,0.240027,0.282384,15.49,14.221356,NaN,NaN,NaN,NaN
1,34833988_4,29807511,amazon,zattoo_amazon_firetv_ultimate_austrian,renewal,Austria,2022-02-20 09:16:19,2022-02-20 09:16:19,2022-03-23 09:16:18,base_ultimate,0.000000,0.000000,0.000000,0.000000,2.0,2.0,2022-03-01,22.0,0.0,0.0,31.0,2.0,8.865054,7.870164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24139431_15,28567399,amazon,zattoo_amazon_firetv_ultimate_austrian,renewal,Austria,2022-02-20 13:25:00,2022-02-20 13:25:00,2022-03-23 13:24:59,base_ultimate,2.998333,3.131579,11.089777,12.491667,2.0,1.0,2022-02-01,9.0,1.0,0.0,31.0,2.0,3.626613,3.219613,0.240027,0.282384,15.49,14.221356,NaN,NaN,NaN,NaN
3,24139431_15,28567399,amazon,zattoo_amazon_firetv_ultimate_austrian,renewal,Austria,2022-02-20 13:25:00,2022-02-20 13:25:00,2022-03-23 13:24:59,base_ultimate,0.000000,0.000000,0.000000,0.000000,2.0,2.0,2022-03-01,22.0,0.0,0.0,31.0,2.0,8.865054,7.870164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,35623652_2,30295233,amazon,zattoo_amazon_firetv_ultimate_austrian_freetrial,new_sale,Austria,2022-02-20 17:25:51,2022-02-20 17:25:51,2022-03-23 17:25:50,base_ultimate,2.998333,3.131579,11.089777,12.491667,2.0,1.0,2022-02-01,9.0,1.0,0.0,31.0,2.0,3.626613,3.219613,0.240027,0.282384,15.49,14.221356,NaN,NaN,NaN,NaN


In [183]:
## export to csv
path = r"/Users/miguelcouto/Desktop/"

mt_df_final.to_csv(os.path.join(path, r"pypayment_v2_amazon_022022.csv"))

In [37]:
mt_df[(mt_df.sku.str.contains('zattoo_web_hiq_swiss_12mo')) & (mt_df.transaction_id == '36948491')]

,transaction_id,zuid,payment_method,sku,type_of_transaction,country_name,transaction_date,total_booking_net_CHF,total_booking_net_EUR,vat_chf,vat_eur,term_start,term_end,product_term_length,product_term_length_months,product_group_finance,revenue_month_number,revenue_month_date,total_revenue_net_EUR,total_revenue_net_CHF,sales_price_eur,sales_price_chf,active_sub_month_end
5296,36948491,3348617,credit_card,zattoo_web_hiq_swiss_12mo,new_sale,Switzerland,2022-05-13 17:06:52,111.420613,107.532247,8.579387,8.279983,2022-05-13 17:06:52,2023-05-13 17:06:52,19,13,base_hiq,1,2022-05-01,5.597569,5.799977,115.81223,120.0,0
5297,36948491,3348617,credit_card,zattoo_web_hiq_swiss_12mo,new_sale,Switzerland,2022-05-13 17:06:52,0.000000,0.000000,0.000000,0.000000,2022-05-13 17:06:52,2023-05-13 17:06:52,30,13,base_hiq,2,2022-06-01,8.838267,9.157859,NaN,NaN,0
5298,36948491,3348617,credit_card,zattoo_web_hiq_swiss_12mo,new_sale,Switzerland,2022-05-13 17:06:52,0.000000,0.000000,0.000000,0.000000,2022-05-13 17:06:52,2023-05-13 17:06:52,31,13,base_hiq,3,2022-07-01,9.132876,9.463121,NaN,NaN,0
5299,36948491,3348617,credit_card,zattoo_web_hiq_swiss_12mo,new_sale,Switzerland,2022-05-13 17:06:52,0.000000,0.000000,0.000000,0.000000,2022-05-13 17:06:52,2023-05-13 17:06:52,31,13,base_hiq,4,2022-08-01,9.132876,9.463121,NaN,NaN,0
5300,36948491,3348617,credit_card,zattoo_web_hiq_swiss_12mo,new_sale,Switzerland,2022-05-13 17:06:52,0.000000,0.000000,0.000000,0.000000,2022-05-13 17:06:52,2023-05-13 17:06:52,30,13,base_hiq,5,2022-09-01,8.838267,9.157859,NaN,NaN,0
5301,36948491,3348617,credit_card,zattoo_web_hiq_swiss_12mo,new_sale,Switzerland,2022-05-13 17:06:52,0.000000,0.000000,0.000000,0.000000,2022-05-13 17:06:52,2023-05-13 17:06:52,31,13,base_hiq,6,2022-10-01,9.132876,9.463121,NaN,NaN,0
5302,36948491,3348617,credit_card,zattoo_web_hiq_swiss_12mo,new_sale,Switzerland,2022-05-13 17:06:52,0.000000,0.000000,0.000000,0.000000,2022-05-13 17:06:52,2023-05-13 17:06:52,30,13,base_hiq,7,2022-11-01,8.838267,9.157859,NaN,NaN,0
5303,36948491,3348617,credit_card,zattoo_web_hiq_swiss_12mo,new_sale,Switzerland,2022-05-13 17:06:52,0.000000,0.000000,0.000000,0.000000,2022-05-13 17:06:52,2023-05-13 17:06:52,31,13,base_hiq,8,2022-12-01,9.132876,9.463121,NaN,NaN,0
5304,36948491,3348617,credit_card,zattoo_web_hiq_swiss_12mo,new_sale,Switzerland,2022-05-13 17:06:52,0.000000,0.000000,0.000000,0.000000,2022-05-13 17:06:52,2023-05-13 17:06:52,31,13,base_hiq,9,2023-01-01,9.132876,9.463121,NaN,NaN,0
5305,36948491,3348617,credit_card,zattoo_web_hiq_swiss_12mo,new_sale,Switzerland,2022-05-13 17:06:52,0.000000,0.000000,0.000000,0.000000,2022-05-13 17:06:52,2023-05-13 17:06:52,28,13,base_hiq,10,2023-02-01,8.249049,8.547335,NaN,NaN,0
